In [ ]:
import traitlets
import ipywidgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg, Robot
import cv2
import numpy as np
import time
from PID import PID

In [ ]:
# -------------------------
# Parameters (tuned for 320x240 cam)
# -------------------------
camera_width  = 320
camera_height = 240

BASE_SPEED    = 0.22         # forward speed (0..1). Start conservative.
STEER_GAIN    = 0.8          # mixes steering into motors
MAX_CMD       = 0.6          # clamp for safety

# PID on normalized error ([-1,1])
# If your PID library expects process_value and has setpoint, we keep setpoint = 0 and feed normalized error.
# (If your PID expects current value vs setpoint, we just use error as "current" and setpoint=0.)
steering_pid = PID(Kp=0.9, Ki=0.0, Kd=0.12, setpoint=0.0)  # solid starting point

# ROI band (bottom of the image)
ROI_Y1_FRACTION = 0.60
ROI_Y2_FRACTION = 0.95

# Loss handling
LOST_MAX = 8  # frames to keep last steering if line lost

# -------------------------
# State
# -------------------------
state = "FOLLOW_LINE"
robot_stop = True
_lost_frames = 0
_last_err_norm = 0.0

def _clip(x, lo=-1.0, hi=1.0):
    return max(lo, min(hi, x))

def _set_motors(left, right):
    left  = _clip(left,  -MAX_CMD, MAX_CMD)
    right = _clip(right, -MAX_CMD, MAX_CMD)
    robot.set_motors(left, right)

def follow_black_line(frame):
    global state, robot_stop, _lost_frames, _last_err_norm

    h, w, _ = frame.shape

    # ----- ROI: look only near the wheels -----
    y1 = int(h * ROI_Y1_FRACTION)
    y2 = int(h * ROI_Y2_FRACTION)
    roi = frame[y1:y2, :]

    # ----- Make black line pop: gray -> blur -> Otsu inverted -----
    gray  = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    blur  = cv2.GaussianBlur(gray, (5,5), 0)
    _, bw = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Morphological cleanup
    kernel = np.ones((5,5), np.uint8)
    bw = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, kernel, iterations=1)
    bw = cv2.morphologyEx(bw, cv2.MORPH_OPEN,  kernel, iterations=1)

    # ----- Find the largest black blob in the ROI -----
    contours, _ = cv2.findContours(bw, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cx_line = None

    if contours:
        cnt = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(cnt)
        if area > 150:  # ignore tiny noise; tune if needed
            M = cv2.moments(cnt)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])          # centroid in ROI coords
                cx_line = cx                           # 0..w-1 within ROI

                # draw for debugging
                cv2.drawContours(roi, [cnt], -1, (0, 255, 0), 2)
                cv2.circle(roi, (cx, (y2 - y1)//2), 5, (0, 0, 255), -1)

    # ----- Compute steering -----
    if cx_line is not None:
        # error: +right, -left (normalize by half-width)
        err = (cx_line - (w / 2)) / (w / 2)   # in [-1, 1]
        # Optional smoothing to reduce jitter
        err = 0.6 * err + 0.4 * _last_err_norm
        _last_err_norm = err

        steering = steering_pid.update(err)   # PID on normalized error; setpoint = 0
        steering = _clip(steering, -1.0, 1.0)

        left_cmd  = BASE_SPEED + (STEER_GAIN * steering)
        right_cmd = BASE_SPEED - (STEER_GAIN * steering)
        _lost_frames = 0
    else:
        # Lost the line: decay towards gentle search turn using last error sign
        _lost_frames += 1
        sign = np.sign(_last_err_norm) if _last_err_norm != 0 else 1.0
        search = 0.12 * sign
        left_cmd  = 0.12 + search
        right_cmd = 0.12 - search
        if _lost_frames > LOST_MAX:
            # stop & wait for re-acquisition
            left_cmd = right_cmd = 0.0

    # Safety stop
    if robot_stop:
        left_cmd = right_cmd = 0.0

    _set_motors(left_cmd, right_cmd)

    # ----- Debug overlay -----
    vis = frame.copy()
    # Draw ROI box
    cv2.rectangle(vis, (0, y1), (w, y2), (255, 0, 0), 2)
    # Show center line
    cv2.line(vis, (w//2, y1), (w//2, y2), (255, 255, 0), 1)
    # Paint ROI mask in place for quick look
    mask_color = cv2.cvtColor(bw, cv2.COLOR_GRAY2BGR)
    vis[y1:y2, :] = cv2.addWeighted(vis[y1:y2, :], 0.7, mask_color, 0.3, 0)
    line_mask_widget.value = bgr8_to_jpeg(vis)


In [ ]:
robot.stop()
camera.stop()